In [1]:
import numpy as np
import pandas as pd
import re, sys, os, csv
from nltk.tokenize import word_tokenize
from collections import Counter

In [ ]:
# get dict of word from english corpus 
def words(text): return re.findall('[a-z]+', text.lower())
dictionary = Counter(words(open('wordlist.txt').read()))
max_word_length = max(map(len, dictionary))
total = float(sum(dictionary.values()))

def word_prob(word): return dictionary[word] / total

In [ ]:
def viterbi_segment(text):
    probs, lasts = [1.0], [0]
    for i in range(1, len(text) + 1):
        prob_k, k = max((probs[j] * word_prob(text[j:i]), j)
                        for j in range(max(0, i - 45), i))
        probs.append(prob_k)
        lasts.append(k)
    words = []
    i = len(text)
    while 0 < i:
        words.append(text[lasts[i]:i])
        i = lasts[i]
    words.reverse()
    return words, probs[-1]

def remove_hashtag(text):
    text = text.group().split(":")[0]
    text = text[1:] 
    try:
        test = int(text[0])
        text = text[1:]
    except:
        pass
    output = ' '.join(viterbi_segment(text)[0])
    #print(output)
    return output

def clean_tweet( tweet):
        tweet = tweet.lower()
        tweet = re.sub("(#[A-Za-z0-9]+)", fix_hashtag, tweet)
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [ ]:
def combine_labels(label):
    if label == "empty":return 0 # neutral
    elif label == "sadness":return 2 # sad
    elif label == "enthusiasm":return 1 # happy
    elif label == "neutral":return 0 # neutral
    elif label == "worry":return 2 # sad
    elif label == "surprise":return 1 # happy
    elif label == "love":return 1 # happy
    elif label == "fun":return 1 # happy
    elif label == "hate":return 3 #hate
    elif label == "happiness":return 1 # happy
    elif label == "boredom":return 0 # neutral
    elif label == "relief":return 1 # happy
    elif label == "anger":return 4 #anger

In [3]:
data_train = pd.read_csv('text_emotion.csv', sep=',')

In [ ]:
dataWriter = csv.writer(open('tweet_data.csv', 'w'), delimiter=',',lineterminator="\n")

total = 40000
for i in range(40000):
    tweet= clean_tweet(data_train.content[i])
    dataWriter.writerow([tweet, str(combine_labels(data_train.sentiment[i]))])
print('Completed preprocessing!')

# complete processing